In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()
train = train.drop([
    "KitchenAbvGr", "KitchenQual", "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu",
    "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea", "GarageQual",
    "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch",
    "ScreenPorch", "PoolArea", "PoolQC", "Fence", "MiscFeature", "MiscVal", "MoSold", "YrSold",
    "SaleType", "SaleCondition"
], axis=1)

test = test.drop([
    "KitchenAbvGr", "KitchenQual", "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu",
    "GarageType", "GarageYrBlt", "GarageFinish", "GarageCars", "GarageArea", "GarageQual",
    "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch",
    "ScreenPorch", "PoolArea", "PoolQC", "Fence", "MiscFeature", "MiscVal", "MoSold", "YrSold",
    "SaleType", "SaleCondition"
], axis=1)
X_train = train[["GrLivArea","FullBath","HalfBath","BedroomAbvGr", "Id"]]
y_train = train["SalePrice"]

X_test = test[["GrLivArea","FullBath","HalfBath","BedroomAbvGr", "Id"]]
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)
X_train = X_train.select_dtypes(include=np.number)
X_test = X_test.select_dtypes(include=np.number)


imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)
numerical_cols = X_train.select_dtypes(include=["int64", "float64"]).columns

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
predicted = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": predictions
     })
predicted.to_csv("predicted.csv", index=False)
def predict_price(grlivarea, fullbath, halfbath, bedrooms):
    # Step 1: Create input DataFrame
    input_data = pd.DataFrame([{
        "GrLivArea": grlivarea,
        "FullBath": fullbath,
        "HalfBath": halfbath,
        "BedroomAbvGr": bedrooms,
        "Id": 0
    }])
    input_data = input_data.select_dtypes(include=np.number)
    input_data = pd.DataFrame(imputer.transform(input_data), columns=input_data.columns)
    input_data[numerical_cols] = scaler.transform(input_data[numerical_cols])
    predicted_price = model.predict(input_data)
    return predicted_price[0]
price = predict_price(2500, 2, 0, 3)
print(f"Predicted Sale Price: {price:.2f}")


Predicted Sale Price: 291477.19
